In [ ]:
#import libraries
import pandas as pd
import numpy as np
import tabula
import camelot


In [ ]:
elk_df = pd.read_excel('data/Elk captures 21-22.xlsx')
bison_df = pd.read_excel('data/Bison_2021_22_Sample sheet.xlsx')

In [ ]:
elk_df.head()

In [ ]:
elk_df.info()

In [ ]:
bison_df.head()

In [ ]:
elk_df.Sex.value_counts()